# Imports

In [1]:
import socket # tcp ip client
import time   # time measurement
import select # timeout tcp ip client recv

# Open TCP port

In [2]:
ip_address = '1.2.3.4'
port = 2000

buffer_size = 64

s=socket.socket(socket.AF_INET, socket.SOCK_STREAM)
#s.setblocking(0)  # set blocking so recv is never blocked
s.connect((ip_address, port))


# Run test cases and close connection

In [3]:
messages6  = ['[SIGN_IN%a]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[SIGN_IN%ilias2017]', 
             '[SIGN_IN%ilias2017]', 
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%a%1;2;11111;22222;33333]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
             '[SIGN_OUT]'
            ]


messages3  = ['[SIGN_IN%a]',
            '[SIGN_IN%ilias2017]', 
            '[SIGN_IN%ilias2017]', 
            '[ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413]',
            '[ORDER%a%1;2;11111;22222;33333]',
            '[SIGN_OUT]'
            ]
messages7  = ['[SIGN_IN%a]',
              '[ORDER%ilias2017%7;1;00244;10342;61130;30534;51221;21413;21413]',
              '[SIGN_IN%ilias2017]', 
              '[SIGN_IN%ilias2017]', 
              '[ORDER%ilias2017%7;1;00244;10342;61130;30534;51221;21413;21413]',
              '[ORDER%a%1;2;11111;22222;33333]',
              '[ORDER%ilias2017%7;1;00244;10342;61130;30534;51221;21413;21413]',
              '[ORDER%ilias2017%7;1;00244;10342;61130;30534;51221;21413;21413]',
              '[ORDER%ilias2017%7;1;00244;10342;61130;30534;51221;21413;21413]',
              '[ORDER%ilias2017%7;1;00244;10342;61130;30534;51221;21413;21413]',
              '[SIGN_OUT]'
            ]

messages8  = ['[SIGN_IN%a]',
              '[ORDER%ilias2017%8;1;00244;10342;61130;30534;51221;21413;21413;21413]',
              '[SIGN_IN%ilias2017]', 
              '[SIGN_IN%ilias2017]', 
              '[ORDER%ilias2017%8;1;00244;10342;61130;30534;51221;21413;21413;21413]',
              '[ORDER%a%1;2;11111;22222;33333]',
              '[ORDER%ilias2017%8;1;00244;10342;61130;30534;51221;21413;21413;21413]',
              '[ORDER%ilias2017%8;1;00244;10342;61130;30534;51221;21413;21413;21413]',
              '[ORDER%ilias2017%8;1;00244;10342;61130;30534;51221;21413;21413;21413]',
              '[ORDER%ilias2017%8;1;00244;10342;61130;30534;51221;21413;21413;21413]',
              '[SIGN_OUT]'
            ]

messages9  = ['[SIGN_IN%a]',
              '[ORDER%ilias2017%9;1;00244;10342;61130;30534;51221;21413;21413;21413;21413]',
              '[SIGN_IN%ilias2017]', 
              '[SIGN_IN%ilias2017]', 
              '[ORDER%ilias2017%9;1;00244;10342;61130;30534;51221;21413;21413;21413;21413]',
              '[ORDER%a%1;2;11111;22222;33333]',
              '[ORDER%ilias2017%9;1;00244;10342;61130;30534;51221;21413;21413;21413;21413]',
              '[ORDER%ilias2017%9;1;00244;10342;61130;30534;51221;21413;21413;21413;21413]',
              '[ORDER%ilias2017%9;1;00244;10342;61130;30534;51221;21413;21413;21413;21413]',
              '[ORDER%ilias2017%9;1;00244;10342;61130;30534;51221;21413;21413;21413;21413]',
              '[SIGN_OUT]'
            ]

messages10  = ['[SIGN_IN%a]',
               '[ORDER%ilias2017%10;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413]',
               '[SIGN_IN%ilias2017]', 
               '[SIGN_IN%ilias2017]', 
               '[ORDER%ilias2017%10;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413]',
               '[ORDER%a%1;2;11111;22222;33333]',
               '[ORDER%ilias2017%10;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%10;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%10;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%10;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413]',
               '[SIGN_OUT]'
            ]

messages11  = ['[SIGN_IN%a]',
               '[ORDER%ilias2017%11;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413]',
               '[SIGN_IN%ilias2017]', 
               '[SIGN_IN%ilias2017]', 
               '[ORDER%ilias2017%11;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413]',
               '[ORDER%a%1;2;11111;22222;33333]',
               '[ORDER%ilias2017%11;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%11;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%11;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%11;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413]',
               '[SIGN_OUT]'
            ]

messages12  = ['[SIGN_IN%a]',
               '[ORDER%ilias2017%12;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413;21413]',
               '[SIGN_IN%ilias2017]', 
               '[SIGN_IN%ilias2017]', 
               '[ORDER%ilias2017%12;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413;21413]',
               '[ORDER%a%1;2;11111;22222;33333]',
               '[ORDER%ilias2017%12;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%12;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%12;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413;21413]',
               '[ORDER%ilias2017%12;1;00244;10342;61130;30534;51221;21413;21413;21413;21413;21413;21413;21413]',
               '[SIGN_OUT]'
            ]

messages20 = ['[SIGN_IN%a]', # sign in wrong
              '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]', #order before sign in
              '[SIGN_IN%ilias2017]', # sign in correct
              '[SIGN_IN%ilias2017]', 
              '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]', # first roder
              '[ORDER%a%1;2;11111;22222;33333]',
              '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]',
              '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]',
              '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]',
              '[ORDER%ilias2017%20;1;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244;00244]',
              '[SIGN_OUT]'
            ]


active_message_list = messages6

# read initial data 
ready = select.select([s],[],[], 3 )
if ready[0]:
    data = s.recv(buffer_size)
else:
    data = "no data received"
print('Connect: <- ' + str(data))
    

for index, message in enumerate(active_message_list):
    #s.send(active_message_list[ index ])
    s.send(active_message_list[index].encode())
    ready = select.select([s],[],[], 4 )
    time.sleep(3)
    if ready[0]:
        data = s.recv(buffer_size)
    else:
        data = "no data received"
    #data.encode('UTF-8')
    print( 'TC' + str(index) + ': -> ' + message + ', <- ' + str(data) + ', length: ' + str(len(data)) )
    time.sleep(1)
s.close()
print( "Connection closed!")

Connect: <- b'*HELLO*'
TC0: -> [SIGN_IN%a], <- b'[SIGN_IN_RS%0%0%PW_WRONG]\n', length: 26
TC1: -> [ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413], <- b'[ORDER_RS%SUCCESSFUL%0%0]\n', length: 26
TC2: -> [SIGN_IN%ilias2017], <- b'[SIGN_IN_RS%1%175%SUCCESSFUL]\n', length: 30
TC3: -> [SIGN_IN%ilias2017], <- b'[SIGN_IN_RS%1%170%SUCCESSFUL]\n', length: 30
TC4: -> [ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413], <- b'[ORDER_RS%SUCCESSFUL%1%165]\n', length: 28
TC5: -> [ORDER%a%1;2;11111;22222;33333], <- b'[ORDER_RS%PW_WRONG%2%340]\n', length: 26
TC6: -> [ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413], <- b'[ORDER_RS%SUCCESSFUL%2%335]\n', length: 28
TC7: -> [ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413], <- b'[ORDER_RS%SUCCESSFUL%3%510]\n', length: 28
TC8: -> [ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413], <- b'[ORDER_RS%SUCCESSFUL%4%685]\n', length: 28
TC9: -> [ORDER%ilias2017%6;1;00244;10342;61130;30534;51221;21413], <- b'[ORDER_RS%ERROR%5%86

# Single tests

In [68]:
message = '[SIGN_OUT]'

s.send(message)
time.sleep(3)
data = s.recv(buffer_size)
print( data)

error: [Errno 10053] An established connection was aborted by the software in your host machine

In [19]:
s.close()